In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, ff_dim, output_dim, dropout=0.1 ):
        super(TransformerModel,self).__init__()
        self.embedding = nn.Embedding(input_dim, model_dim)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 100, model_dim))
        self.encoder = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead= num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(model_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.fc_out(x[:, -1, :])  # Output from the last token
        return x